In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/blenderrrrrrrr/blend_submission26_2.csv
/kaggle/input/blenderrrrrrrr/blend_submission26_1.csv
/kaggle/input/blenderrrrrrrr/blend_submission26_3.csv
/kaggle/input/top-files/blend_submission23.csv
/kaggle/input/top-files/blend_submission25.csv
/kaggle/input/top-files/blend_submission24 (1).csv
/kaggle/input/submissionfiles/submission6 (3) (1).csv
/kaggle/input/submissionfiles/submission4 (1) (1).csv
/kaggle/input/submissionfiles/submission (6) (1).csv
/kaggle/input/taxi-fare-guru-total-amount-prediction-challenge/sample.csv.csv
/kaggle/input/taxi-fare-guru-total-amount-prediction-challenge/train.csv
/kaggle/input/taxi-fare-guru-total-amount-prediction-challenge/test.csv
/kaggle/input/stackerrrr/blend_submission26_4.csv
/kaggle/input/stackerrrr/blend_submission26_5.csv
/kaggle/input/stackerrrr/blend_submission26_6.csv
/kaggle/input/xgboost/submission (7).csv
/kaggle/input/stackinngggggggg/blend_submission21.csv
/kaggle/input/stackinngggggggg/blend_submission2 (1).csv


In [ ]:
!pip uninstall mlnotify -y

In [2]:
train_data = pd.read_csv("/kaggle/input/taxi-fare-guru-total-amount-prediction-challenge/train.csv")
test_data = pd.read_csv("/kaggle/input/taxi-fare-guru-total-amount-prediction-challenge/test.csv")
samp_submission = pd.read_csv("/kaggle/input/taxi-fare-guru-total-amount-prediction-challenge/sample.csv.csv")

In [ ]:
train_data.head()

In [ ]:
test_data.head()

In [ ]:
train_data.shape

In [ ]:
test_data.shape

In [ ]:
samp_submission

In [ ]:
import pandas as pd

# Assuming 'train_data' is your DataFrame containing the training set

# Select only numerical columns
numerical_columns = train_data.select_dtypes(include='number').columns

# Create a summary table
summary_data = pd.DataFrame(index=numerical_columns, columns=['Nulls', 'Positives', 'Negatives', 'Unknowns', 'Min', 'Max', 'Needs Scaling'])

# Fill in the summary table
for column in numerical_columns:
    summary_data.loc[column, 'Nulls'] = train_data[column].isnull().sum()
    summary_data.loc[column, 'Positives'] = (train_data[column] > 0).sum()
    summary_data.loc[column, 'Negatives'] = (train_data[column] < 0).sum()
    summary_data.loc[column, 'Unknowns'] = (train_data[column] == '?').sum()
    summary_data.loc[column, 'Min'] = train_data[column].min()
    summary_data.loc[column, 'Max'] = train_data[column].max()
    
    # You can customize this condition based on your criteria for scaling
    summary_data.loc[column, 'Needs Scaling'] = 'Yes' if (train_data[column].max() > 1 or train_data[column].min() < 0) else 'No'

# Display the summary table
print(summary_data)

In [3]:
traindata_filtered = train_data[train_data['passenger_count'] != 0]

In [ ]:
traindata_filtered.head()

In [4]:
# Convert the datetime columns to datetime objects
traindata_filtered['tpep_pickup_datetime'] = pd.to_datetime(traindata_filtered['tpep_pickup_datetime'])
traindata_filtered['tpep_dropoff_datetime'] = pd.to_datetime(traindata_filtered['tpep_dropoff_datetime'])

# Calculate the total time taken and create a new column 'total_time'
traindata_filtered['total_time'] = traindata_filtered['tpep_dropoff_datetime'] - traindata_filtered['tpep_pickup_datetime']


traindata_filtered['total_time_minutes'] = traindata_filtered['total_time'].dt.total_seconds() / 60

# Drop the original datetime columns
traindata_filtered = traindata_filtered.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime'], axis=1)

# Print the resulting DataFrame
traindata_filtered

/tmp/ipykernel_32/4088963902.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  traindata_filtered['tpep_pickup_datetime'] = pd.to_datetime(traindata_filtered['tpep_pickup_datetime'])
/tmp/ipykernel_32/4088963902.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  traindata_filtered['tpep_dropoff_datetime'] = pd.to_datetime(traindata_filtered['tpep_dropoff_datetime'])
/tmp/ipykernel_32/4088963902.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

,VendorID,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,extra,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,total_time,total_time_minutes
0,1,1.0,2.14,1.0,N,120,9,Credit Card,2.5,7.165589,0.0,1.0,20.64,2.5,0.00,-1 days +23:14:24,-45.600000
1,0,1.0,2.70,1.0,N,15,215,Credit Card,3.5,6.067401,0.0,1.0,25.55,2.5,0.00,-1 days +22:56:34,-63.433333
2,1,1.0,1.15,1.0,N,167,223,Credit Card,0.0,4.111547,0.0,1.0,17.64,2.5,0.00,0 days 00:53:39,53.650000
3,0,1.0,0.40,1.0,N,128,239,Credit Card,2.5,6.411079,0.0,1.0,12.80,2.5,0.00,0 days 00:56:52,56.866667
4,1,3.0,1.10,1.0,N,203,52,Credit Card,1.0,4.769377,0.0,1.0,18.00,2.5,0.00,0 days 00:18:50,18.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174995,1,3.0,3.45,1.0,N,147,167,Credit Card,1.0,8.732495,0.0,1.0,28.08,2.5,0.00,-1 days +23:31:25,-28.583333
174996,1,1.0,9.44,1.0,N,154,191,Cash,5.0,0.283275,0.0,1.0,59.95,2.5,1.75,0 days 01:01:24,61.400000
174997,0,1.0,2.40,1.0,N,168,106,Credit Card,2.5,4.245354,0.0,1.0,33.50,2.5,0.00,0 days 01:10:02,70.033333
174998,1,1.0,4.71,1.0,N,240,100,Credit Card,2.5,10.479776,0.0,1.0,40.80,2.5,0.00,-1 days +23:21:38,-38.366667


In [ ]:
traindata_filtered.head()

In [5]:
traindata_filtered['total_time_minutes'] = traindata_filtered['total_time_minutes'].abs()

In [6]:
traindata_filtered = traindata_filtered[traindata_filtered['trip_distance'] < 500]
traindata_filtered=traindata_filtered[traindata_filtered['tip_amount'] < 200]
traindata_filtered['payment_type'].replace('unknown', 'Credit Card', inplace = True)

In [7]:
traindata_filtered = pd.get_dummies(traindata_filtered, columns=['payment_type'], dtype=int)
traindata_filtered = pd.get_dummies(traindata_filtered, columns=['store_and_fwd_flag'],dtype=int)

In [ ]:
traindata_filtered

In [8]:
X = traindata_filtered.drop(['total_amount','total_time' ], axis=1)

In [9]:
Y = traindata_filtered['total_amount']

In [ ]:
X

In [10]:
X = X.fillna(X.mean())

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

In [ ]:
testdata_filtered

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import StackingRegressor, VotingRegressor, ExtraTreesRegressor, RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
import xgboost as xgb
from tqdm import tqdm



# Split the data into training and testing sets
# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

extra_trees_model = ExtraTreesRegressor(n_estimators=200, max_depth=30, min_samples_split=7, random_state=42)
# xgb_model = XGBRegressor(n_estimators=300, max_depth=5, learning_rate=0.1, random_state=42)


# stacked_model = StackingRegressor(
#     estimators=[('extra_trees', extra_trees_model), ('xgb', xgb_model)],
#     final_estimator=XGBRegressor(random_state=42),
#     cv = 5
# )

In [12]:
extra_trees_model.fit(X,Y)

ExtraTreesRegressor(max_depth=30, min_samples_split=7, n_estimators=200,
                    random_state=42)

In [17]:
test_data_sorted = test_data
testdata_filtered = test_data_sorted
# Convert the datetime columns to datetime objects
testdata_filtered['tpep_pickup_datetime'] = pd.to_datetime(testdata_filtered['tpep_pickup_datetime'])
testdata_filtered['tpep_dropoff_datetime'] = pd.to_datetime(testdata_filtered['tpep_dropoff_datetime'])

# Calculate the total time taken and create a new column 'total_time'
testdata_filtered['total_time'] = testdata_filtered['tpep_dropoff_datetime'] - testdata_filtered['tpep_pickup_datetime']


testdata_filtered['total_time_minutes'] = testdata_filtered['total_time'].dt.total_seconds() / 60
testdata_filtered['total_time_minutes'] = testdata_filtered['total_time_minutes'].abs()
# Drop the original datetime columns
testdata_filtered = testdata_filtered.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime','total_time'], axis=1)

# Print the resulting DataFrame
testdata_filtered

,VendorID,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,extra,tip_amount,tolls_amount,improvement_surcharge,congestion_surcharge,Airport_fee,total_time_minutes
0,1,1.0,4.95,1.0,N,20,3,Credit Card,1.0,6.067612,0.0,1.0,2.5,0.00,4.000000
1,1,1.0,2.10,1.0,N,9,81,Credit Card,2.5,6.191269,0.0,1.0,2.5,0.00,8.500000
2,1,1.0,0.95,1.0,N,92,90,Cash,2.5,3.983872,0.0,1.0,2.5,0.00,66.733333
3,0,1.0,0.80,1.0,N,19,102,Credit Card,3.5,6.839341,0.0,1.0,2.5,0.00,12.450000
4,1,1.0,4.01,1.0,N,131,229,Cash,0.0,1.468943,0.0,1.0,0.0,0.00,28.383333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,1,1.0,0.61,1.0,N,48,72,Credit Card,2.5,4.043482,0.0,1.0,2.5,0.00,48.616667
49996,1,1.0,5.02,1.0,N,57,103,Cash,1.0,2.895882,0.0,1.0,0.0,1.75,14.750000
49997,1,1.0,1.92,1.0,N,150,78,Credit Card,1.0,5.636590,0.0,1.0,2.5,0.00,39.116667
49998,0,1.0,5.40,1.0,N,4,154,Credit Card,3.5,7.293144,0.0,1.0,2.5,0.00,26.750000


In [18]:
testdata_filtered['payment_type'].replace('unknown', 'Credit Card', inplace = True)

In [19]:
testdata_filtered = pd.get_dummies(testdata_filtered, columns=['payment_type'],dtype=int)
testdata_filtered = pd.get_dummies(testdata_filtered, columns=['store_and_fwd_flag'],dtype=int)
testdata_filtered = testdata_filtered.fillna(testdata_filtered.mean())

In [20]:
testdata_filtered

,VendorID,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,extra,tip_amount,tolls_amount,improvement_surcharge,congestion_surcharge,Airport_fee,total_time_minutes,payment_type_Cash,payment_type_Credit Card,payment_type_UPI,payment_type_Wallet,store_and_fwd_flag_N,store_and_fwd_flag_Y
0,1,1.0,4.95,1.0,20,3,1.0,6.067612,0.0,1.0,2.5,0.00,4.000000,0,1,0,0,1,0
1,1,1.0,2.10,1.0,9,81,2.5,6.191269,0.0,1.0,2.5,0.00,8.500000,0,1,0,0,1,0
2,1,1.0,0.95,1.0,92,90,2.5,3.983872,0.0,1.0,2.5,0.00,66.733333,1,0,0,0,1,0
3,0,1.0,0.80,1.0,19,102,3.5,6.839341,0.0,1.0,2.5,0.00,12.450000,0,1,0,0,1,0
4,1,1.0,4.01,1.0,131,229,0.0,1.468943,0.0,1.0,0.0,0.00,28.383333,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,1,1.0,0.61,1.0,48,72,2.5,4.043482,0.0,1.0,2.5,0.00,48.616667,0,1,0,0,1,0
49996,1,1.0,5.02,1.0,57,103,1.0,2.895882,0.0,1.0,0.0,1.75,14.750000,1,0,0,0,1,0
49997,1,1.0,1.92,1.0,150,78,1.0,5.636590,0.0,1.0,2.5,0.00,39.116667,0,1,0,0,1,0
49998,0,1.0,5.40,1.0,4,154,3.5,7.293144,0.0,1.0,2.5,0.00,26.750000,0,1,0,0,1,0


In [21]:
test_predictions = extra_trees_model.predict(testdata_filtered)

In [22]:
testdata_filtered['total_amount'] = test_predictions

In [23]:
testdata_filtered.reset_index(drop=True, inplace=True)

In [24]:
testdata_filtered['ID'] = range(1, len(testdata_filtered) + 1)
dummy_data = testdata_filtered[['ID','total_amount']]
submission_data = dummy_data[['ID', 'total_amount']]
submission_data.to_csv('submission_extratrees.csv', index=False)

In [ ]:
import pandas as pd

# Load your multiple submission files
# submission1 = pd.read_csv('/kaggle/input/blenderrrrrrrr/blend_submission26_3.csv')
submission2 = pd.read_csv('/kaggle/input/top-files/blend_submission24 (1).csv')
submission3 = pd.read_csv('/kaggle/working/blend_submission26_4.csv')

# Combine predictions using a simple average
blend_predictions = (0.8* submission2['total_amount'] + 0.2 * submission3['total_amount'])

# Create a new submission file
blend_submission = pd.DataFrame({'ID': submission1['ID'], 'total_amount': blend_predictions})
blend_submission.to_csv('blend_submission26_6.csv', index=False)

In [28]:
import pandas as pd

# Load your multiple submission files
submission3 = pd.read_csv('/kaggle/input/stackerrrr/blend_submission26_6.csv')
submission4 = pd.read_csv('/kaggle/input/top-files/blend_submission24 (1).csv')

# Combine predictions using a simple average
blend_predictions = (0.5*submission3['total_amount']+ 0.5*submission4['total_amount'])

# Create a new submission file
blend_submission = pd.DataFrame({'ID': submission1['ID'], 'total_amount': blend_predictions})
blend_submission.to_csv('blend_submission32.csv', index=False)# Assuming you have multiple prediction files (CSV files)
# submission1 = pd.read_csv('/kaggle/input/stackerrrr/blend_submission26_4.csv')
# submission2 = pd.read_csv('/kaggle/input/stackerrrr/blend_submission26_5.csv')
# submission3 = pd.read_csv('/kaggle/input/stackerrrr/blend_submission26_6.csv')
# submission4 = pd.read_csv('/kaggle/input/top-files/blend_submission24 (1).csv')

# submission_new = 0.2*submission1 + 0.2 * submission2 + 0.2*submission3 + 0.4* submission4
# submission_new.to_csv('new.csv', index=False)

In [ ]:
stacking_data

In [ ]:
data1 = pd.read_csv('/kaggle/working/new_submission.csv')

In [ ]:
testdata_filtered